In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import bifacial_radiance
bifacial_radiance.__version__

'0.4.1+121.g952f921.dirty'

In [2]:
testfolder = r'TEMP\SquareDesign'
if not os.path.exists(testfolder):
    os.makedirs(testfolder)


In [3]:
rawfile = r'..\..\SAM_SP_WeatherFile.csv'
demo = bifacial_radiance.RadianceObj("SouthPole", path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.7)
metdata = demo.readWeatherFile(rawfile, source='SAM') 

path = TEMP\SquareDesign
Loading albedo, 1 value(s), 0.700 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


C:\Users\sayala\Documents\GitHub\pvlib-python\pvlib\spa.py:1200: RuntimeWarning: invalid value encountered in remainder
  H0 = np.degrees(np.arccos(cos_arg)) % 180
C:\Users\sayala\Documents\GitHub\pvlib-python\pvlib\spa.py:1211: RuntimeWarning: invalid value encountered in remainder
  m[1] = m[1] % 1
C:\Users\sayala\Documents\GitHub\pvlib-python\pvlib\spa.py:1212: RuntimeWarning: invalid value encountered in remainder
  m[2] = m[2] % 1
C:\Users\sayala\Documents\GitHub\pvlib-python\pvlib\spa.py:1229: RuntimeWarning: invalid value encountered in remainder
  Hp = (vs + lon - alpha_prime) % 360


In [4]:
timestamp = metdata.datetime.index(pd.to_datetime('2021-12-24 13:0:0 -0'))
demo.gendaylit(timestamp) 

# For sanity check, we are creating the same module but with different names for each orientation.
numpanels=1 
ygap = 0.01 # m Spacing between modules on each shed.
y=2   # m. module size, one side
x=1   # m. module size, other side. for landscape, x > y
xgap = 2
mymodule = demo.makeModule(name='module',y=y,x=x, numpanels=numpanels, xgap=xgap, ygap=ygap)


Module Name: module
Module module updated in module.json


In [5]:
nPanels = 2400
print("System Size ", nPanels*.380, "kW")
nMods = 20 # 7         20
nRows = 30 # 11        30
pitch = 4
xgap = 3
z = 0
x = 1
CO = 47          # Center Offset space
goal = 240                   
innerR = xgap+pitch

print("Mega Array Side: ", nMods*z + (nRows-1)*pitch + nMods*x + (nMods-1) * xgap+ CO,  "m")

System Size  912.0 kW
Mega Array Side:  240 m


In [6]:
Rays = 4
clearance_height = 0.5

sceneObjDict = {}
for rr in range(0, Rays):
    ray_azm = 45 + 360*rr/Rays
    sazm = ray_azm - 45
    centerpitch = innerR + pitch*(nRows-1)/2         # This might be off by half a pitch, check for even and odd nMods  
    azm = ray_azm - 90  # Radiance and Pvlib use N = 0, so converting
    centerpitch_x = centerpitch*np.cos(np.radians(azm))
    centerpitch_y = centerpitch*np.sin(np.radians(azm))*-1
    sceneDict = {'tilt':90,'pitch':pitch,'clearance_height':clearance_height,'azimuth':sazm, 'nMods': nMods, 'nRows': nRows, 
             'appendRadfile':True, 'originx': centerpitch_x, 'originy': centerpitch_y} 
    sceneObjDict['sceneObj'+str(rr)] = demo.makeScene(mymodule, sceneDict)  

In [7]:
octfile = demo.makeOct() 

Created SouthPole.oct


In [8]:
!rvu -vf views\front.vp -e .01 -pe 0.3 -vp 5 -50 50 -vd 0 0.7071 -0.7071 SouthPole.oct

In [ ]:
#demo.genCumSky()
#octfile = demo.makeOct(demo.getfilelist()) 

In [10]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo.basename)  #

In [ ]:
# Make a color render and falsecolor image of the scene.
analysis.makeImage('side.vp')
analysis.makeFalseColor('side.vp')

In [ ]:
### Analysis

In [11]:
for obj in sceneObjDict:
    for mm in range(1, 2):#nMods+1):
        for rr in range(1, 2):# nRows+1):
            frontscan, backscan = analysis.moduleAnalysis(sceneObjDict[obj], sensorsy=4, sensorsx = 4, modWanted = mm, rowWanted = rr)
            frontdict, backdict = analysis.analysis(octfile, obj, frontscan, backscan)  # compare the back vs front irradiance  

Linescan in process: sceneObj0_Row1_Module1_Front
Linescan in process: sceneObj0_Row1_Module1_Back
Saved: results\irr_sceneObj0_Row1_Module1.csv
Linescan in process: sceneObj1_Row1_Module1_Front
Linescan in process: sceneObj1_Row1_Module1_Back
Saved: results\irr_sceneObj1_Row1_Module1.csv
Linescan in process: sceneObj2_Row1_Module1_Front
Linescan in process: sceneObj2_Row1_Module1_Back
Saved: results\irr_sceneObj2_Row1_Module1.csv
Linescan in process: sceneObj3_Row1_Module1_Front
Linescan in process: sceneObj3_Row1_Module1_Back
Saved: results\irr_sceneObj3_Row1_Module1.csv


In [14]:
bifaciality = 0.7

filetitle = 'irr_sceneObj0_Mod_1_Row_3_Row3_Module1.csv'
for obj in sceneObjDict:
    for mm in range(1, nMods+1):
        for rr in range(1, nRows+1):
            filetitle = 'results\irr_'+obj+'_Row'+str(rr)+'_Module'+str(mm)+'.csv'
            data=bifacial_radiance.load.read1Result(filetitle)
            Gpoa = data['Wm2Front'].mean() + data['Wm2Back'].mean()*bifaciality


FileNotFoundError: [Errno 2] No such file or directory: 'results\\irr_sceneObj0_Row2_Module1.csv'

In [ ]:
sensorsy=4  # 1 per module. consider increasing the number but be careful with sensors in the space between modules.
analysis = bifacial_radiance.AnalysisObj(octfile, demo.basename)  
frontscan, backscan = analysis.moduleAnalysis(sceneObj1, sensorsy=sensorsy)
frontdict, backdict = analysis.analysis(octfile, "EastFacingShed", frontscan, backscan)  # compare the back vs front irradiance  

frontscan, backscan = analysis.moduleAnalysis(sceneObj2, sensorsy=sensorsy )
frontdict2, backdict2 = analysis.analysis(octfile, "WestFacingShed", frontscan, backscan)  # compare the back vs front irradiance  
